In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import expipe
import pathlib
import numpy as np
import sys
sys.path.append('../ca2-mec')
import data_processing as dp
#import septum_mec.analysis.data_processing as dp
from trackunitmulticomparison import TrackMultipleSessions
#from septum_mec.analysis.registration import store_notebook
import re
import joblib
import multiprocessing
import shutil
import psutil
import pandas as pd
import matplotlib.pyplot as plt
import quantities as pq
import exdir
from distutils.dir_util import copy_tree
from datetime import timedelta
from tqdm import tqdm_notebook as tqdm
#from septum_mec.analysis.trackunitmulticomparison import TrackMultipleSessions
import networkx as nx
#from nxpd import draw
%matplotlib inline

In [3]:
project_path = dp.project_path()

project = expipe.get_project(project_path)
actions = project.actions

In [4]:
output = pathlib.Path('output/identify_neurons')
output.mkdir(parents=True, exist_ok=True)

In [5]:
identify_neurons = project.require_action('identify-neurons')

In [6]:
actions['133-090720-4'].attributes

{'registered': '2020-07-09T19:50:36',
 'datetime': '2020-07-09T18:47:06',
 'type': 'Recording',
 'tags': ['open-ephys', 'experiment'],
 'entities': ['133'],
 'users': ['sarah'],
 'location': 'ibv',
 'data': {'main': 'actions\\133-090720-4\\data\\main.exdir'}}

In [7]:
data_loader = dp.Data()

In [8]:
skip_actions = [

]

In [9]:
sessions = []
for action in actions.values():
    if action.id in skip_actions:
        continue
    if action.type != 'Recording':
        continue
    action_data_path = dp.get_data_path(action)
    exdir_object =  exdir.File(action_data_path)
    if 'processing' not in exdir_object:
        continue
        
    processing = exdir_object['processing']
    
    if not 'electrophysiology' in processing:
        continue
    elphys = processing['electrophysiology']
    if 'spikesorting' not in elphys:
        continue  

    sessions.append({
       'action': action.id,
       'session': int(action.id.split('-')[-1]),
       'entity': int(action.entities[0]),

   })
sessions = pd.DataFrame(sessions)

In [10]:
sessions

,action,session,entity
0,133-010720-2,2,133
1,133-010720-3,3,133
2,133-020720-1,1,133
3,133-020720-2,2,133
4,133-020720-3,3,133
...,...,...,...
75,136-070720-2,2,136
76,136-080720-1,1,136
77,136-080720-2,2,136
78,136-080720-3,3,136


In [11]:
sessions['date'] = sessions.apply(lambda x: x.action.split('-')[1], axis=1)
sessions['entity_date'] = sessions.apply(lambda x: '-'.join(x.action.split('-')[:2]), axis=1)

In [12]:
sessions.to_csv(output / 'sessions.csv', index=False)

# Identify unique neurons

In [13]:
include = ["133-090720-4",
          "133-090720-5",
          "133-090720-6",
          "133-090720-7"]

In [14]:
sessions_include = sessions.query("action.isin(@include)")

In [15]:
sessions_include

,action,session,entity,date,entity_date
28,133-090720-4,4,133,090720,133-090720
29,133-090720-5,5,133,090720,133-090720
30,133-090720-6,6,133,090720,133-090720
31,133-090720-7,7,133,090720,133-090720


In [19]:
# save graphs
for entity, values in sessions_include.groupby('entity'):
    data_path = output / f'{entity}-graphs'
    if data_path.exists():
        continue
    print('Processing', entity)
    unit_matching = TrackMultipleSessions(
        actions, values.action.values.tolist(), 
        progress_bar=tqdm, verbose=False, data_path=data_path
    )
    unit_matching.do_matching()
    unit_matching.make_graphs_from_matches()
#    unit_matching.compute_time_delta_edges()
    unit_matching.compute_depth_delta_edges()
    # save graph with all dissimilarities for later use
    unit_matching.save_graphs()

Processing 133


In [17]:
data_path

WindowsPath('output/identify_neurons/133-graphs')

# Plot comparisons

In [20]:
unit_comp = unit_matching

In [138]:
unit_comp = TrackMultipleSessions(actions, data_path=r'output\identify_neurons\133-graphs')

In [25]:
unit_comp.load_graphs()

In [26]:
max_dissimilarity = .05
max_depth_delta = 100

unit_comp.remove_edges_above_threshold('weight', max_dissimilarity)
unit_comp.remove_edges_above_threshold('depth_delta', max_depth_delta)

In [27]:
unit_comp.remove_edges_with_duplicate_actions()
unit_comp.identify_units()

In [28]:
unit_comp.identified_units

{0: {'3ac2cf08-bfe8-4970-91dd-6236f03d6dcf': {'average_dissimilarity': None,
   'original_unit_ids': defaultdict(list, {'133-090720-4': ['1096']})},
  'c59bdb8b-aa65-4bdc-8367-e911be2bebdb': {'average_dissimilarity': None,
   'original_unit_ids': defaultdict(list, {'133-090720-4': ['1099']})},
  '7a3846d3-9df8-4033-b731-83f81c381d1d': {'average_dissimilarity': None,
   'original_unit_ids': defaultdict(list, {'133-090720-4': ['1124']})},
  'c58d8f82-7283-4332-9048-87b3b1820fb1': {'average_dissimilarity': None,
   'original_unit_ids': defaultdict(list, {'133-090720-4': ['1147']})},
  'dfeca869-15ee-4515-bc33-0081158d00c5': {'average_dissimilarity': None,
   'original_unit_ids': defaultdict(list, {'133-090720-4': ['1398']})},
  '1fd6d428-bdfb-49c4-9682-14fa247c55b2': {'average_dissimilarity': None,
   'original_unit_ids': defaultdict(list, {'133-090720-4': ['1401']})},
  '6c88d03a-39ae-4a66-98ae-5c83f285c8d3': {'average_dissimilarity': None,
   'original_unit_ids': defaultdict(list, {'133

In [29]:
unit_comp.plot_matches('template', chan_group=0, step_color=False)
plt.tight_layout()

ValueError: '61' is not in list

<Figure size 720x4536 with 0 Axes>

# Store uniqe unit ids to csv

In [120]:
max_dissimilarity = .05
max_depth_delta = 100
id_nums = {}
num = 0
for entity, values in sessions_include.groupby('entity'):
    data_path = output / f'{entity}-graphs'
    unit_matching = TrackMultipleSessions(
        actions, values.action.values.tolist(), 
        progress_bar=tqdm, verbose=False, data_path=data_path
    )
    unit_matching.load_graphs()
    # cutoff large dissimilarities
    unit_matching.remove_edges_above_threshold('weight', max_dissimilarity)
    unit_matching.remove_edges_above_threshold('depth_delta', max_depth_delta)
    unit_matching.remove_edges_with_duplicate_actions()
    unit_matching.identify_units()
    units = []
    for ch, group in unit_matching.identified_units.items():
        for unit_id, val in group.items():
            for action_id, orig_unit_ids in val['original_unit_ids'].items():
                if unit_id not in id_nums:
                    id_nums[unit_id] = num
                    num += 1
                units.extend([
                    {
                        'unit_name': name, 
                        'unit_idnum': id_nums[unit_id],
                        'unit_id': unit_id, 
                        'action': action_id,
                        'channel_group': ch,
                        'max_dissimilarity': max_dissimilarity,
                        'max_depth_delta': max_depth_delta
                    } 
                    for name in orig_unit_ids])
    units = pd.DataFrame(units)
    units = units[~units.action.isin(skip_actions)]
    units.to_csv(output / f'{entity}-units.csv', index=False)

ConstructorError: could not determine a constructor for the tag 'tag:yaml.org,2002:python/object/apply:numpy.core.multiarray.scalar'
  in "output\identify_neurons\133-graphs\graph-group-0.yaml", line 959, column 14

In [45]:
unique_units = pd.concat([
    pd.read_csv(p) 
    for p in output.iterdir() 
    if p.name.endswith('-units.csv')])

In [ ]:
# remove 1834-010319-3 unit 72

In [46]:
unique_units.to_csv(output / 'units.csv', index=False)

# Store results in Expipe action

In [47]:
identify_neurons.data['sessions'] = 'sessions.csv'
identify_neurons.data['units'] = 'units.csv'

In [48]:
copy_tree(output, str(identify_neurons.data_path()))

['/media/storage/expipe/septum-mec/actions/identify-neurons/data/1834-units.csv',
 '/media/storage/expipe/septum-mec/actions/identify-neurons/data/all_non_identified_units.csv',
 '/media/storage/expipe/septum-mec/actions/identify-neurons/data/1834-graphs/graph-group-7.yaml',
 '/media/storage/expipe/septum-mec/actions/identify-neurons/data/1834-graphs/graph-group-0.yaml',
 '/media/storage/expipe/septum-mec/actions/identify-neurons/data/1834-graphs/graph-group-6.yaml',
 '/media/storage/expipe/septum-mec/actions/identify-neurons/data/1834-graphs/graph-group-1.yaml',
 '/media/storage/expipe/septum-mec/actions/identify-neurons/data/1834-graphs/graph-group-2.yaml',
 '/media/storage/expipe/septum-mec/actions/identify-neurons/data/1834-graphs/graph-group-5.yaml',
 '/media/storage/expipe/septum-mec/actions/identify-neurons/data/1834-graphs/graph-group-3.yaml',
 '/media/storage/expipe/septum-mec/actions/identify-neurons/data/1834-graphs/graph-group-4.yaml',
 '/media/storage/expipe/septum-mec/act

In [96]:
store_notebook(
    identify_neurons, "00-identify-neurons.ipynb")